# Welcome to Python Notebooks and Blender! 👋

This course teaches you all of the tools you need automate repetitive reporting -- starting from the basics: Jupyter, Python and Mito. 

The course focuses on building Python skills that will be helpful to you in practice, leaving behind a lot of the theoretical explanations you might find in other courses. 

The goal of this course is to demistify the Python basics so you have the confidence to start applying Python automations in your day-to-day work.

Things to know before we start:
* 💡 -> the lightbulb emoji means thers an **important information**.
* 🧑‍💻 -> the Person Behind Computer emoji means thers an **task** waiting for you.
* This is not an exam, it's an opportunity to learn something new. You will get stuck at some point -- don't hesitate to ask questions to the Mito team, your colleagues, or Stack Overflow.
* Most imporantly: Enjoy the joureny!

## Objects in Blender


In [1]:
import bpy

💡 The below code will select the default cube and create a variable

In [ ]:
cube = bpy.context.active_object
print(cube)

Now, we display attributes of the cube into the output.   
🧑‍💻 Run `cube.location` , `cube.scale` and `len(cube.data.vertices)`.  
💡 After writing `cube.` followed by the "Tab" key you will get suggestions from auto complete.  


In [ ]:
# TODO replace me with your code

🧑‍💻  Now, manipulate the cube.
1. Set the z location with `cube.location.z = 2`
2. Scale the cube with `cube.scale = (1, 1, 2)`
3. 

In [ ]:
# TODO replace me with your code


💡 We can also add default objects (torus, monkey, plane, ...) and select them with `obj = bpy.context.active_object`

In [ ]:
bpy.ops.mesh.primitive_monkey_add(size=2.5, location=(0, 0, 3))
monkey = bpy.context.active_object

bpy.ops.mesh.primitive_torus_add(major_radius=2, minor_radius=0.3, location=(0,0, 2))
tours = bpy.context.active_object

bpy.ops.mesh.primitive_plane_add(size=3, location=(0, 0, 1.1))  
plane = bpy.context.active_object

💡 Objects can also be removed

In [ ]:
bpy.data.objects.remove(tours)

 🧑‍💻 Now, delete the monkey and the plane as well.

💡 The below script will randomly place 10 spheres in the scene.

In [ ]:
import random

for i in range(10):
    x = random.uniform(-10, 10)
    y = random.uniform(-10, 10)
    z = 0
    bpy.ops.mesh.primitive_uv_sphere_add(radius=1, location=(x, y, z))

 🧑‍💻 Now, run the above cell 5 times.

💡 Executing the cell multiple times will add objects multiple times.   
  Therefore, it's good to clear the whole scene from time to time.

🚨 The below cell will delete everything from your scene.

In [2]:
def fresh_scene():
    # Deselect and delete all objects except cameras and lights
    bpy.ops.object.select_all(action='DESELECT')
    for obj in bpy.context.scene.objects:
        if obj.type not in {'CAMERA', 'LIGHT'}:
            obj.select_set(True)
    bpy.ops.object.delete()

fresh_scene()

💡 Objects will show up in our scene collection. We can also give them names.

In [ ]:
fresh_scene()
cube_names = ["Foo", "Bar", "Baz", "Qux"]
for i, name in enumerate(cube_names):
    bpy.ops.mesh.primitive_cube_add(location=(i * 3, 0, 0))  
    cube = bpy.context.object                                     
    cube.name = name                        

💡 And we can apply modifiers to change the appearance.


In [ ]:
cube_names = ["Foo", "Bar", "Baz", "Qux"]

for name in cube_names:
    cube = bpy.data.objects.get(name)
    if cube:   # Check if the cube exists
        bevel_modifier = cube.modifiers.new(name="Bevel", type='BEVEL')
        bevel_modifier.width = 0.5
        bevel_modifier.segments = 3

💡 Running the above cell multiple times will add modifiers on top of each other and should be avoided.  
🧑‍💻 Try changing the bevel width in the below cell for two values.

In [ ]:
for name in cube_names:
    cube = bpy.data.objects.get(name)
    if cube: # Check if the cube exists
        for modifier in cube.modifiers:
            if modifier.type == 'BEVEL':
                modifier.width = 0.2

💡 Finally, modifiers can also be removed like this:

In [ ]:
cube_names = ["Foo", "Bar", "Baz", "Qux"]

for name in cube_names:
    cube = bpy.data.objects.get(name)
    if cube: # Check if the cube exists
        cube.modifiers.clear()

 🧑‍💻 Now make a simple histogram using 20 cylinders.
* The cylinders should be be placed on the x axis.
* the Height should be a random value between 3 and 5  
  
 **Hint**: use `bpy.ops.mesh.primitive_cylinder_add(radius=0.5, depth=2, location=(0,1,0))` to add a cylinder

In [ ]:
# TODO replace me with your code


In [ ]:
# TODO ONLY IN TUTOR VERSION
fresh_scene()
for i in range(0,7):
    z = random.uniform(4, 10)
    bpy.ops.mesh.primitive_cylinder_add(radius=0.5, depth=z, location=(0,i,z/2))

💡 We can also create a collection and put the objects into it.

In [ ]:
fresh_scene()

# Create the Histogram collection
histo_collection = bpy.data.collections.new("Histogram")
bpy.context.scene.collection.children.link(histo_collection)

# Create 5 spheres, name them, and link each to the Histogram collection
for i in range(5):
    bpy.ops.mesh.primitive_uv_sphere_add(radius=0.5, location=(i * 2, 0, 0))
    sphere = bpy.context.object
    sphere.name = f"Sphere_{i+1}"
    histo_collection.objects.link(sphere)
    
    # Unlink the sphere from all other collections
    for collection in sphere.users_collection:
        if collection != histo_collection:
            collection.objects.unlink(sphere)

💡 Delete the Histogram collection and all objects within it

In [ ]:
# Delete the Histogram collection and all objects within it
bpy.data.collections.remove(histo_collection)

# Using Python Packages

Let's use Python packages to generate data.  
💡 We can install packages using uv

In [3]:
!uv pip install numpy

Using Python 3.11.10 environment at /Users/jan-hendrik/Desktop/blender_python_workshop/.venv
Audited 1 package in 7ms


💡 We can import a package and make a gaussian mesh

In [43]:
import numpy as np
sigma = 2
x = np.linspace(-5, 5, 20)
y = np.linspace(-5, 5, 20)
x, y = np.meshgrid(x, y)
z = 2 * np.exp(-(x**2 + y**2) / (2 * sigma**2))

💡 We can now add sphere by sphere, but that's slow

In [44]:
fresh_scene()
# Flatten the arrays and use zip to iterate over coordinates
for xi, yi, zi in zip(x.flatten(), y.flatten(), z.flatten()):
    bpy.ops.mesh.primitive_uv_sphere_add(radius=0.1, location=(xi, yi, zi))

💡 Instead we can use a Mesh, and save verices in a point cloud!

In [46]:
fresh_scene()

import mathutils
# Flatten the arrays and combine into a list of vectors
points = [mathutils.Vector((xi, yi, zi)) for xi, yi, zi in zip(x.flatten(), y.flatten(), z.flatten())]

# Create a new mesh and object for the point cloud
mesh = bpy.data.meshes.new("PointCloudMesh")
point_obj = bpy.data.objects.new("PointCloud", mesh)

# Apply the points to the mesh
mesh.from_pydata(points, edges=[], faces=[])
mesh.update()
bpy.context.collection.objects.link(point_obj)

Here, we have a wave field:

In [64]:
fresh_scene()

amplitude = 1 

# Generate x and y arrays
x = np.linspace(-10, 10, 150)
y = np.linspace(-10, 10, 150)
x, y = np.meshgrid(x, y)
z = amplitude * np.sin (x) * np.cos(y) 

# Flatten the arrays and combine into a list of vectors
points = [mathutils.Vector((xi, yi, zi)) for xi, yi, zi in zip(x.flatten(), y.flatten(), z.flatten())]

# Create a new mesh and object for the point cloud
mesh = bpy.data.meshes.new("PointCloudMesh")
point_obj = bpy.data.objects.new("PointCloud", mesh)

# Apply the points to the mesh
mesh.from_pydata(points, edges=[], faces=[])
mesh.update()
bpy.context.collection.objects.link(point_obj)

In [65]:
amplitude = 3
z = amplitude * np.sin(x) * np.cos(y)
points = [mathutils.Vector((xi, yi, zi)) for xi, yi, zi in zip(x.flatten(), y.flatten(), z.flatten())]

# Update the existing mesh with the new points
mesh.clear_geometry()  # Clear existing geometry
mesh.from_pydata(points, edges=[], faces=[])
mesh.update()

In [72]:
import pandas as pd
url = 'https://simplemaps.com/static/data/country-cities/de/de.csv'
df = pd.read_csv(url)
df


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Berlin,52.5200,13.4050,Germany,DE,Berlin,primary,4890363.0,3755251.0
1,Stuttgart,48.7775,9.1800,Germany,DE,Baden-Württemberg,admin,2787724.0,632865.0
2,Munich,48.1375,11.5750,Germany,DE,Bavaria,admin,2606021.0,1512491.0
3,Hamburg,53.5500,10.0000,Germany,DE,Hamburg,admin,2484800.0,1892122.0
4,Cologne,50.9364,6.9528,Germany,DE,North Rhine-Westphalia,NaN,1084831.0,1084831.0
...,...,...,...,...,...,...,...,...,...
124,Altbach,48.7239,9.3797,Germany,DE,Baden-Württemberg,NaN,6422.0,6422.0
125,Merzhausen,47.9664,7.8286,Germany,DE,Baden-Württemberg,NaN,5347.0,5347.0
126,Buckenhof,49.5939,11.0500,Germany,DE,Bavaria,NaN,3157.0,3157.0
127,Offenbach,50.1006,8.7665,Germany,DE,Hesse,minor,NaN,NaN


In [73]:
selected_columns = df[['lat', 'lng', 'population_proper']]
print(selected_columns)

         lat      lng  population_proper
0    52.5200  13.4050          3755251.0
1    48.7775   9.1800           632865.0
2    48.1375  11.5750          1512491.0
3    53.5500  10.0000          1892122.0
4    50.9364   6.9528          1084831.0
..       ...      ...                ...
124  48.7239   9.3797             6422.0
125  47.9664   7.8286             5347.0
126  49.5939  11.0500             3157.0
127  50.1006   8.7665                NaN
128  50.9781  11.0289                NaN

[129 rows x 3 columns]


In [68]:
!uv pip install pandas

Using Python 3.11.10 environment at /Users/jan-hendrik/Desktop/blender_python_workshop/.venv
Audited 1 package in 2ms


In [ ]:
1. render within notebook
2. 

# Now it's time to add Geometry nodes!